# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
CitiesData = "../WeatherPy/output_data/cities.csv"
CitiesDataDF = pd.read_csv(CitiesData)
CitiesDataDF.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,villa maria,-32.41,-63.24,46.00,44,0,4.00,AR
1,dourbali,11.81,15.86,73.15,92,99,6.78,TD
2,katsuura,35.13,140.30,97.00,66,20,8.05,JP
3,promissao,-21.54,-49.86,72.32,45,27,5.32,BR
4,bredasdorp,-34.53,20.04,46.40,87,0,9.17,ZA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = CitiesDataDF[["Lat", "Lon"]].astype(float)
Humidityy = CitiesDataDF["Humidity"].astype(float)
len(locations)

547

In [5]:
Figure = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidityy,dissipating=False, max_intensity=100,
                                 point_radius = 1)
Figure.add_layer(heat_layer)

Figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = CitiesDataDF.loc[(CitiesDataDF["Max Temp"] < 80) & (CitiesDataDF["Max Temp"] > 70) \
                                   & (CitiesDataDF["Wind Speed"] < 10)]

In [7]:

narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country
1,dourbali,11.81,15.86,73.15,92,99,6.78,TD
3,promissao,-21.54,-49.86,72.32,45,27,5.32,BR
7,ca mau,9.18,105.15,79.30,84,100,5.35,VN
13,albany,42.60,-73.97,73.00,68,0,3.87,US
14,rumonge,-3.97,29.44,72.70,43,82,2.37,BI
...,...,...,...,...,...,...,...,...
528,pombia,45.65,8.63,75.00,83,20,4.70,IT
531,bulungu,-4.55,18.60,78.75,57,54,2.37,CD
533,lokoja,7.80,6.74,72.64,93,100,5.21,NG
534,san carlos,9.66,-68.58,74.52,93,85,2.84,VE


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lon,Hotel Name
1,dourbali,TD,11.81,15.86,
3,promissao,BR,-21.54,-49.86,
7,ca mau,VN,9.18,105.15,
13,albany,US,42.60,-73.97,
14,rumonge,BI,-3.97,29.44,
...,...,...,...,...,...
528,pombia,IT,45.65,8.63,
531,bulungu,CD,-4.55,18.60,
533,lokoja,NG,7.80,6.74,
534,san carlos,VE,9.66,-68.58,


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000,
    "types": "lodging",
    "key": g_key}
results = []
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lon"]
    
    params["locations"] = f"{lat}, {lng}"
    cityprint = row["City"]

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {cityprint}.")
    HotelResponse = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = HotelResponse["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        
#     try:
#         print(f"The closest hotel in the city: {City} is {results[0]['name']}.")
        
#         types_df.loc[index, 'name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

Retrieving Results for Index 1: dourbali.
Missing field/result... skipping.
Retrieving Results for Index 3: promissao.
Missing field/result... skipping.
Retrieving Results for Index 7: ca mau.
Missing field/result... skipping.
Retrieving Results for Index 13: albany.
Missing field/result... skipping.
Retrieving Results for Index 14: rumonge.
Missing field/result... skipping.
Retrieving Results for Index 25: westport.
Missing field/result... skipping.
Retrieving Results for Index 26: ribeira grande.
Missing field/result... skipping.
Retrieving Results for Index 38: toora-khem.
Missing field/result... skipping.
Retrieving Results for Index 43: dillon.
Missing field/result... skipping.
Retrieving Results for Index 60: port macquarie.
Missing field/result... skipping.
Retrieving Results for Index 62: amapa.
Missing field/result... skipping.
Retrieving Results for Index 72: puerto lempira.
Missing field/result... skipping.
Retrieving Results for Index 83: coolum beach.
Missing field/result.

In [14]:
HotelResponse

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
